<a href="https://colab.research.google.com/github/4nuragb/PGM/blob/main/Markov_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [80]:
!pip install yfinance
!pip install talib-binary

In [81]:
import numpy as np
import pandas as pd
# import tensorflow as tf
import yfinance as yf
import talib as ta

In [82]:
SPY= yf.download("SPY",start="2018-01-01",end="2019-12-31")

[*********************100%***********************]  1 of 1 completed


In [83]:
data=SPY[['Open','High','Low','Adj Close']].copy()
data.tail()

,Open,High,Low,Adj Close
Date,,,,
2019-12-23,321.589996,321.649994,321.059998,310.219543
2019-12-24,321.470001,321.519989,320.899994,310.229126
2019-12-26,321.649994,322.950012,321.640015,311.880646
2019-12-27,323.739990,323.799988,322.279999,311.803314
2019-12-30,322.950012,323.100006,320.549988,310.084259


In [84]:
data['pct_ret']=data['Adj Close'].pct_change()
data.tail()

,Open,High,Low,Adj Close,pct_ret
Date,,,,,
2019-12-23,321.589996,321.649994,321.059998,310.219543,0.001528
2019-12-24,321.470001,321.519989,320.899994,310.229126,0.000031
2019-12-26,321.649994,322.950012,321.640015,311.880646,0.005324
2019-12-27,323.739990,323.799988,322.279999,311.803314,-0.000248
2019-12-30,322.950012,323.100006,320.549988,310.084259,-0.005513


In [85]:
data['state']=data['pct_ret'].apply(lambda x:'Up' if(x>0.001) 
else ('Down' if (x<-0.001)
else 'Flat'))

In [86]:
data.tail()

,Open,High,Low,Adj Close,pct_ret,state
Date,,,,,,
2019-12-23,321.589996,321.649994,321.059998,310.219543,0.001528,Up
2019-12-24,321.470001,321.519989,320.899994,310.229126,0.000031,Flat
2019-12-26,321.649994,322.950012,321.640015,311.880646,0.005324,Up
2019-12-27,323.739990,323.799988,322.279999,311.803314,-0.000248,Flat
2019-12-30,322.950012,323.100006,320.549988,310.084259,-0.005513,Down


In [87]:
data['priorstate']=data['state'].shift()
data.tail()

,Open,High,Low,Adj Close,pct_ret,state,priorstate
Date,,,,,,,
2019-12-23,321.589996,321.649994,321.059998,310.219543,0.001528,Up,Up
2019-12-24,321.470001,321.519989,320.899994,310.229126,0.000031,Flat,Up
2019-12-26,321.649994,322.950012,321.640015,311.880646,0.005324,Up,Flat
2019-12-27,323.739990,323.799988,322.279999,311.803314,-0.000248,Flat,Up
2019-12-30,322.950012,323.100006,320.549988,310.084259,-0.005513,Down,Flat


In [88]:
#Frequency Distributions 
states= data[['priorstate','state']].dropna()
states_mat=states.groupby(['priorstate','state']).size().unstack()
states_mat

state,Down,Flat,Up
priorstate,,,
Down,77,12,97
Flat,33,10,30
Up,77,50,115


In [89]:
#Initial transition matrix 
transition_matrix=states_mat.apply(lambda x: x/float(x.sum()), axis=1)
transition_matrix

state,Down,Flat,Up
priorstate,,,
Down,0.413978,0.064516,0.521505
Flat,0.452055,0.136986,0.410959
Up,0.318182,0.206612,0.475207


In [90]:
t0=transition_matrix.copy()
t1=round(t0.dot(t0),4)
t1

state,Down,Flat,Up
priorstate,,,
Down,0.3665,0.1433,0.4902
Flat,0.3798,0.1328,0.4873
Up,0.3763,0.1470,0.4767


In [91]:
#Find the transition matrix at t2
t2=round(t0.dot(t1),4)
t2

state,Down,Flat,Up
priorstate,,,
Down,0.3725,0.1446,0.4830
Flat,0.3723,0.1434,0.4843
Up,0.3739,0.1429,0.4832


In [92]:
#Find the transition matrix at t3 
# t3= round(t0.dot(t2),10)
t3=t0.dot(t2)
t3

state,Down,Flat,Up
priorstate,,,
Down,0.373217,0.143636,0.483188
Flat,0.373048,0.143737,0.483260
Up,0.373124,0.143544,0.483364


In [93]:
pd.DataFrame(np.linalg.matrix_power(t0,4))

,0,1,2
0,0.373216,0.143620,0.483164
1,0.373048,0.143718,0.483233
2,0.373132,0.143531,0.483337


In [94]:
#Find the equilibrium matrix 
i=1
a=t0.copy()
b=t0.dot(t0)
while(not(a.equals(b))):
  print("Iteration number: "+str(i))
  i+=1
  a=b.copy()
  b=b.dot(t0)

Iteration number: 1
Iteration number: 2
Iteration number: 3
Iteration number: 4
Iteration number: 5
Iteration number: 6
Iteration number: 7
Iteration number: 8
Iteration number: 9
Iteration number: 10
Iteration number: 11
Iteration number: 12
Iteration number: 13
Iteration number: 14
Iteration number: 15
Iteration number: 16
Iteration number: 17
Iteration number: 18
Iteration number: 19
Iteration number: 20
